# 如何取消执行

```{=mdx}
:::info 前提条件

本指南假定您熟悉以下概念：

- [LangChain 表达式语言](/docs/concepts/lcel)
- [链（Chains）](/docs/how_to/sequence/)
- [流式传输（Streaming）](/docs/how_to/streaming/)

:::
```

在构建长时间运行的链或 [LangGraph](https://langchain-ai.github.io/langgraphjs/) 代理时，您可能希望在某些情况下中断执行，例如用户离开您的应用程序或提交新查询。

[LangChain 表达式语言 (LCEL)](/docs/concepts/lcel) 通过运行时 [signal](https://developer.mozilla.org/zh-CN/docs/Web/API/AbortController/signal) 选项支持中止正在进行的可运行对象。

```{=mdx}
:::caution 兼容性

内置的信号支持需要 `@langchain/core>=0.2.20`。有关升级指南，请参见[此处](/docs/how_to/installation/#installing-integration-packages)。

:::
```

**注意：** 个别集成（如聊天模型或检索器）可能在中止执行的实现上存在缺失或差异。本指南中描述的信号支持将适用于链的各个步骤之间。

要查看其工作原理，可以构建一个如下的链，该链执行[检索增强生成](/docs/tutorials/rag)。它通过首先使用 [Tavily](/docs/integrations/retrievers/tavily) 在网络上搜索，然后将结果传递给聊天模型以生成最终答案来回答问题：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs />
```

In [1]:
// @lc-docs-hide-cell
import { ChatAnthropic } from "@langchain/anthropic";

const llm = new ChatAnthropic({
  model: "claude-3-5-sonnet-20240620",
});

In [2]:
import { TavilySearchAPIRetriever } from "@langchain/community/retrievers/tavily_search_api";
import type { Document } from "@langchain/core/documents";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables";

const formatDocsAsString = (docs: Document[]) => {
  return docs.map((doc) => doc.pageContent).join("\n\n")
}

const retriever = new TavilySearchAPIRetriever({
  k: 3,
});

const prompt = ChatPromptTemplate.fromTemplate(`
Use the following context to answer questions to the best of your ability:

<context>
{context}
</context>

Question: {question}`)

const chain = RunnableSequence.from([
  {
    context: retriever.pipe(formatDocsAsString),
    question: new RunnablePassthrough(),
  },
  prompt,
  llm,
  new StringOutputParser(),
]);

如果以正常方式调用它，可以看到它返回的是最新的信息：

In [3]:
await chain.invoke("what is the current weather in SF?");

Based on the provided context, the current weather in San Francisco is:

Temperature: 17.6°C (63.7°F)
Condition: Sunny
Wind: 14.4 km/h (8.9 mph) from WSW direction
Humidity: 74%
Cloud cover: 15%

The information indicates it's a sunny day with mild temperatures and light winds. The data appears to be from August 2, 2024, at 17:00 local time.


现在，让我们尽早中断它。初始化一个 [`AbortController`](https://developer.mozilla.org/zh-CN/docs/Web/API/AbortController)，并将其 `signal` 属性传入链式执行中。为了说明取消操作会尽可能快地发生，我们设置一个 100 毫秒的超时：

In [4]:
const controller = new AbortController();

const startTimer = console.time("timer1");

setTimeout(() => controller.abort(), 100);

try {
  await chain.invoke("what is the current weather in SF?", {
    signal: controller.signal,
  });
} catch (e) {
  console.log(e);
}

console.timeEnd("timer1");

Error: Aborted
    at EventTarget.<anonymous> (/Users/jacoblee/langchain/langchainjs/langchain-core/dist/utils/signal.cjs:19:24)
    at [nodejs.internal.kHybridDispatch] (node:internal/event_target:825:20)
    at EventTarget.dispatchEvent (node:internal/event_target:760:26)
    at abortSignal (node:internal/abort_controller:370:10)
    at AbortController.abort (node:internal/abort_controller:392:5)
    at Timeout._onTimeout (evalmachine.<anonymous>:7:29)
    at listOnTimeout (node:internal/timers:573:17)
    at process.processTimers (node:internal/timers:514:7)
timer1: 103.204ms


你可以看到执行在超过100毫秒后结束。查看[this LangSmith 跟踪](https://smith.langchain.com/public/63c04c3b-2683-4b73-a4f7-fb12f5cb9180/r)，你会发现模型从未被调用。

## 流式传输

在流式传输时，你也可以传递一个 `signal`。这使你能够更精确地控制在 `for await...of` 循环中使用 `break` 语句来取消当前运行，该操作仅在最终输出已经开始流式传输后才会触发。以下示例使用了 `break` 语句——请注意取消操作发生前经过的时间：

In [5]:
const startTimer2 = console.time("timer2");

const stream = await chain.stream("what is the current weather in SF?");

for await (const chunk of stream) {
  console.log("chunk", chunk);
  break;
}

console.timeEnd("timer2");

chunk 
timer2: 3.990s


现在将其与使用信号进行比较。请注意，你需要将流包装在 `try/catch` 块中：

In [6]:
const controllerForStream = new AbortController();

const startTimer3 = console.time("timer3");

setTimeout(() => controllerForStream.abort(), 100);

try {
  const streamWithSignal = await chain.stream("what is the current weather in SF?", {
    signal: controllerForStream.signal
  });
  for await (const chunk of streamWithSignal) {
    console.log(chunk);
    break;
  } 
} catch (e) {
  console.log(e);  
}

console.timeEnd("timer3");

Error: Aborted
    at EventTarget.<anonymous> (/Users/jacoblee/langchain/langchainjs/langchain-core/dist/utils/signal.cjs:19:24)
    at [nodejs.internal.kHybridDispatch] (node:internal/event_target:825:20)
    at EventTarget.dispatchEvent (node:internal/event_target:760:26)
    at abortSignal (node:internal/abort_controller:370:10)
    at AbortController.abort (node:internal/abort_controller:392:5)
    at Timeout._onTimeout (evalmachine.<anonymous>:7:38)
    at listOnTimeout (node:internal/timers:573:17)
    at process.processTimers (node:internal/timers:514:7)
timer3: 100.684ms


## 相关

- [将参数从一个步骤传递到下一个步骤](/docs/how_to/passthrough)
- [派发自定义事件](/docs/how_to/callbacks_custom_events)